# Named Entity Recognition using Flair on CONLL-2003
## Experiment description
This notebook contains a ML fabric flow for Named Entity Recognition
using the [flair NLP package](https://github.com/flairNLP/flair/)

Note: This example implements all the required data objects. For a clean notebook which uses the objects already implemented in the `ner_sample` Python package [click here](flair_ner_clean.ipynb).

##### Jupyter helpers:

In [ ]:
%reload_ext autoreload
%autoreload

Define imports

In [2]:
from pathlib import Path
from typing import List

import flair
import requests
import torch
from flair.data import Corpus, Sentence
from flair.datasets import CONLL_03
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, PooledFlairEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from seqeval.metrics import f1_score, accuracy_score
from tqdm.notebook import tqdm

from ner_sample import ExperimentRunner
from ner_sample.data import DataLoader
from ner_sample.evaluation import Evaluator, EvaluationMetrics
from ner_sample.experimentation import MlflowExperimentation
from ner_sample.models import BaseModel


In [ ]:
class ConllDataLoader(DataLoader):

    def __init__(self, dataset_name, dataset_version='1', local_data_path = '../data/processed/'):
        self.folds = ('eng.train', 'eng.testa', 'eng.testb')
        self.local_data_path = local_data_path
        super().__init__(dataset_name=dataset_name, dataset_version=dataset_version)

    def download_dataset(self) -> None:
        
        if self.dataset_name=="conll_03" and self.dataset_version == '1':
            
            for fold in self.folds:
                local_path = Path(self.local_data_path,self.dataset_name).resolve()
                
                if not local_path.exists():
                    local_path.mkdir(parents=True)

                dataset_file = Path(local_path, fold)
                if dataset_file.exists():
                    print("Dataset already exists, skipping download")
                    return

                dataset_path=f"https://raw.githubusercontent.com/glample/tagger/master/dataset/{fold}"
                response = requests.get(dataset_path)
                dataset_raw = response.text
                with open(dataset_file, "w") as f:
                    f.write(dataset_raw)
                print(f"Finished writing fold {fold} to {self.local_data_path}")

            print(f"Finished downloading dataset {self.dataset_name} version {self.dataset_version}")
        
        else:
            raise ValueError("Selected dataset was not found")

    def get_dataset(self) -> Corpus:
        try:
            return CONLL_03(base_path=self.local_data_path, in_memory=True)

        except FileNotFoundError:
            print(f"Dataset {self.dataset_name} with version {self.dataset_version} not found in data/raw")


## Load data
*replace MyDataLoader with your DataLoader implementation*

In [ ]:
data_loader = ConllDataLoader(dataset_name = "conll_03")
data_loader.download_dataset()
corpus = data_loader.get_dataset()
corpus=corpus.downsample(0.05) # Just for example purposes
corpus

In [ ]:
print(f"First sample in train sample:\n {corpus.train.dataset[0]}")

Define experimentation object, which will be used for logging the experiments parameters, metrics and artifacts
*Replace MlflowExperimentation if you use a different experimentation system*

In [ ]:
experimentation = MlflowExperimentation()

Create model/logic:

In [ ]:
flair.device = torch.device("cpu")

class FlairNERModel(BaseModel):

    def __init__(self, 
                 corpus: Corpus,
                 hidden_size: int =256, 
                 pooling: str = 'min', 
                 word_embeddings: str='glove',
                 train_with_dev: bool = True,
                 max_epochs: int = 10):
        self.tag_type = 'ner'
        self.tag_dictionary = None
        self.tagger = None
        self.embeddings = None

        self.hidden_size = hidden_size
        self.pooling = pooling
        self.word_embeddings = word_embeddings
        self.train_with_dev = train_with_dev
        self.max_epochs = max_epochs
        
        self.set_tagger_definition(corpus)
        
        hyper_params = self.get_hyper_params(hidden_size=hidden_size, 
                             pooling=pooling, 
                             word_embeddings=word_embeddings, 
                             train_with_dev=train_with_dev, 
                             max_epochs=max_epochs)
        
        super().__init__(**hyper_params)

    def get_hyper_params(self, **hyper_params):
        basic_params = {param_name: param_value 
                        for (param_name, param_value) in self.tagger.__dict__.items() 
                        if type(param_value) in (bool, float, int, str)}
        hyper_params.update(basic_params)
        return hyper_params
        
    def set_embeddings_definition(self) -> List[TokenEmbeddings]:
        """
        Sets the embedding layers used by this tagger
        """
        # initialize embeddings
        embedding_types: List[TokenEmbeddings] = [

        # Word embeddings (default = GloVe)
        WordEmbeddings(self.word_embeddings),

        # contextual string embeddings, forward
        PooledFlairEmbeddings('news-forward', pooling=self.pooling),

        # contextual string embeddings, backward
        PooledFlairEmbeddings('news-backward', pooling=self.pooling)
        ]
        self.embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)
    
    def set_tagger_definition(self, corpus:Corpus) -> SequenceTagger:
        """
        Returns the definition of the Flair SequenceTagger (the full model)
        :param corpus: Used only for setting the tag_dictionary
        """
        
        if not self.embeddings:
            self.set_embeddings_definition()
        self.tag_dictionary = corpus.make_tag_dictionary(tag_type=self.tag_type)
        
        tagger: SequenceTagger = SequenceTagger(hidden_size=self.hidden_size,
                                                embeddings=self.embeddings,
                                                tag_dictionary=self.tag_dictionary,
                                                tag_type=self.tag_type,
                                                use_crf=False)
        self.tagger = tagger
        
    def fit(self, corpus: Corpus) -> None:
        # initialize trainer
        trainer: ModelTrainer = ModelTrainer(self.tagger, corpus)

        trainer.train('models/taggers/flair-ner',
                      train_with_dev=self.train_with_dev,  
                      max_epochs=self.max_epochs)

    def predict(self, sentences):
        tagged_sentences = []
        for sentence in tqdm(sentences):
            self.tagger.predict(sentence)
            tagged_sentences.append(sentence)
        return tagged_sentences
            
model = FlairNERModel(corpus=corpus)

In [ ]:
TRAIN=False

if TRAIN:
    model.fit(corpus)
else:
    # Simulate training has finished by downloading a pretrained model
    model.tagger = SequenceTagger.load('ner')

Example prediction

In [ ]:
example_sentence = Sentence("In Penny Lane, there is a barber showing photographs")

model.predict([example_sentence])
for token in example_sentence.tokens:
    #print(token.__dict__)
    print(f" {token.text} | {token.get_tag('ner')}")

In [ ]:
for sentence in [corpus.test[i] for i in range(5)]:
    [token.add_tag_label("gold_ner",token.get_tag("ner")) for token in sentence.tokens]
    [token.set_label("ner",value="O") for token in sentence.tokens]
#for token in test1:
    

for token in corpus.test[1]:
    print(token.get_tag("ner"))


In [ ]:
corpus.test

In [ ]:
for sentence in [corpus.test[i] for i in range(5)]:
    [token.add_tag_label("gold_ner",token.get_tag("ner")) for token in sentence.tokens]
    [token.set_label("ner",value="O") for token in sentence.tokens]

predictions = model.predict(corpus.test)

Define evaluation

In [ ]:
class NEREvaluationMetrics:
    """
    This class holds the metrics calculated during the experiment run
    """
    def __init__(self,f1, accuracy):
        self.f1 = f1
        self.accuracy = accuracy

class NEREvaluator(Evaluator):
    """
    This class holds the logic for evaluating a prediction outcome
    """
    def evaluate(self, y_test, predicted_sentences) -> EvaluationMetrics:
        golds = []
        predicted = []
        print(y_test)
        print(predicted_sentences)
        for sentence in predicted_sentences:

            gold_tags = [token.get_tag('gold_ner').value for token in sentence.tokens]
            golds.append(gold_tags)
            predicted_tags = [token.get_tag('ner').value for token in sentence.tokens]
            predicted.append(predicted_tags)
        
        f1 = f1_score(golds, predicted)
        accuracy = accuracy_score(golds, predicted)
        return NEREvaluationMetrics(f1=f1,accuracy=accuracy)
        
evaluator = NEREvaluator()

Define experiment runner

In [ ]:
class NERExperimentRunner(ExperimentRunner):
    
    def __init__(self,
                model,
                corpus,
                data_loader,
                log_experiment,
                experiment_logger,
                evaluator,
                experiment_name):
        self.corpus = corpus
        super().__init__(model=model,
                         data_loader=data_loader, 
                         log_experiment=log_experiment,
                         experiment_logger=experiment_logger,
                         evaluator=evaluator, 
                         experiment_name=experiment_name,
                         X_train=None, X_test=None)
    
    def predict(self):
        """
        Overwriting the ExperimentRunner predict to verify that tags are marked correctly before prediction
        """
        corpus = self.corpus
        # Copy gold NER to new label and assign O to all ner labels (to be populated during inference)
        for sentence in corpus.test:
            [token.add_tag_label("gold_ner",token.get_tag("ner")) for token in sentence.tokens]
            [token.set_label("ner",value="O") for token in sentence.tokens]

        self._predictions = self.model.predict(corpus.test)


experiment_runner = NERExperimentRunner(
    model=model,
    corpus=corpus,
    data_loader=data_loader,
    log_experiment=True,
    experiment_logger=experimentation,
    evaluator=evaluator,
    experiment_name="Experiment",
)

results = experiment_runner.evaluate()
print(results)
